In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Data Preprocessing

In [3]:
df_raw = pd.read_csv('train_test.csv')

In [4]:
select_cols = [col for col in df_raw.columns if col not in ('charttime','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','los_icu','text_embeddings','los_icu_class')]

In [5]:
df_full = df_raw[select_cols].groupby('id').transform(lambda x: x.fillna(x.mean()))

In [6]:
df = df_full.fillna(df_full.mean())

# Prepare data for training

In [7]:
feature_cols = df.columns.drop(['icu_death'])
target = 'icu_death'
X = df[feature_cols]
y = df[target]

# numerical columns
num = df.select_dtypes(include=['float']).columns
X_num = X[num]

# min-max standardization
scaler = MinMaxScaler(feature_range=(0, 1))
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=num, index=X_num.index)
X_scaled = X.copy()
X_scaled[num] = X_num_scaled[num]

time_steps = 25
X_scaled = np.array([X_scaled[i:i + time_steps] for i in range(0, len(X_scaled), time_steps)])
y = np.array([y[i] for i in range(time_steps - 1, len(y), time_steps)])

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Modeling

In [8]:
# Construction
model = Sequential()
model.add(LSTM(units=25, return_sequences=False, input_shape=(time_steps, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

d:\Anaconda\envs\tensorflow\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy') # , metrics=[tf.keras.metrics.F1Score()]

In [10]:
# calculate weight of each classification
total_samples = len(y_train)
class_count = np.bincount(y_train)
class_weights = total_samples / (len(class_count) * class_count)
# change into dictionary format
class_weights_dict = dict(enumerate(class_weights))

In [11]:
class_weights_dict

{0: 0.5592341956484496, 1: 4.720535068691251}

In [12]:
# fit model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), class_weight=class_weights_dict)

Epoch 1/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.6779 - val_loss: 0.8359
Epoch 2/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.5907 - val_loss: 0.3412
Epoch 3/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.5775 - val_loss: 0.4848
Epoch 4/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.5493 - val_loss: 0.5846
Epoch 5/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.5486 - val_loss: 0.5890
Epoch 6/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.5200 - val_loss: 0.6785
Epoch 7/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.5174 - val_loss: 0.4279
Epoch 8/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.5120 - val_loss: 0.5883
Epoch 9/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.5300 - val_loss: 0.4208
Epoch 10/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.5145 - val_loss: 0.5022
Epoch 11/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.4992 - val_loss: 0.4650
Epoch 12/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step

In [13]:
# save the trained model
model.save('lstm_mortality.keras')

In [14]:
# on testing set
y_test_pred = model.predict(X_test)

# classify based on probability
y_pred_class = (y_test_pred > 0.5).astype(int)

# evaluate the result
precision = precision_score(y_test, y_pred_class)
recall = recall_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [15]:
# predict the whole X
y_pred = model.predict(X_scaled)

# classify based on probability
y_pred_class = (y_pred > 0.5).astype(int)

# evaluate the result
precision = precision_score(y, y_pred_class)
recall = recall_score(y, y_pred_class)
f1 = f1_score(y, y_pred_class)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [16]:
result_df = pd.DataFrame(y_pred, columns=['predictions'])

# output death probability of all patients
result_df.to_csv('lstm_probs_of_death_traintest.csv', index=False)